In [594]:

from datetime import datetime  # calculating time until expiration
import numpy as np  # performing numerical methods
import pandas as pd  # printing dataframes
import yfinance as yf # retrieving financial data

In [637]:

# parent/super class
"""
Pricing of stock futures and construction of a portfolio containing both underlying equity
and futures as a hedge.

Attributes
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

ticker: str
    ticker of underlying stock

spot_price: float
    spot price of underlying stock, in united states dollars $

initial_time: str
    current date

final_time: str
    expiration date

dividend: float
    dividend yield, in percentage %, default = 0.00%

risk_tolerance: float
    risk tolerance, in percentage %, default = 0.00%

cap: float
    user capital, in united states dollars $, default = $10,000

shares: float
    user position in underlying stock, in united states dollars $, default = 100 * s
year_time: date
    proportion of diff in time 
liquid: float
    Liquid/capital money (user inputted)
    

"""

class Edward_Dashboard:
    def __init__(self, ticker: str, final_time: str, r_tol: float = 0.00,liquid: float=1000, cap: float = 10000, shares: float = 100):
        assert (1 <= len(ticker) <= 5),  'ticker symbol must have between one and five characters'
        assert r_tol >= 0, 'risk tolerance cannot be less than 0.00%'
        assert cap >= 10000,  'user must have at least $10,000 in capital'
        assert shares >= 0.00, 'user must have at least 100 shares in underlying equity'
        self.ticker_symbol = ticker
        # ticker symbol(user inputted )
        self.ticker = yf.Ticker(ticker.upper()).info
        self.tickerObj =yf.Ticker(ticker.upper())
        # spot price
        s = float(self.ticker['regularMarketPrice'])
        assert s >= 0.00,  'stock price cannot be less than $0.00, cannot use this stock'
        self.spot_price = s
    
        # intial time(not user inputted )
        current_date = datetime.now()
        time = f"{current_date.year}-{current_date.month}-{current_date.day}"
        self.initial_time = datetime.strptime(time, "%Y-%m-%d")
        # final time(user inputted)
        self.final_time = datetime.strptime(final_time, "%Y-%m-%d")
        print(self.final_time)
        # porpotion of diff in time
        self.year_time = (self.final_time - self.initial_time).days/365
        print(self.year_time)
        # risk tolerance
        self.risk_tolerance = float(r_tol)
        # liquid/captial money(user inputted)
        self.liquid = float(liquid)
        # total value of shares (user inputted)
        self.shares = float(shares * self.spot_price)
        #total capital
        self.cap = float(cap)
    def get_historical_data(self):
        temp_csv_str = f"{self.ticker_symbol}.csv"
        temp_csv = self.tickerObj.history(period ="max").to_csv(temp_csv_str)
        new_history_df = pd.read_csv(temp_csv_str)
        historical_data_close= new_history_df[["Date","Close"]]
        return historical_data_close
    def div_rate(self):
        #this is the dividend rate from the info of the function
        #this is the dividend rate from the most recent year
        dividend_list = self.tickerObj.dividends.to_list()
        if len(dividend_list) != 0:
            return dividend_list[-1]
        
    def risk_free_rate(self): # calculate risk-free rate of return
        rates = pd.read_csv('rates.csv')

        if 0 < self.year_time <= 1 / 12:
            nominal = rates['1 Mo'].mean()
        elif 1 / 12 < self.year_time <= 1 / 6:
            nominal = rates['2 Mo'].mean()
        elif 1 / 6 < self.year_time <= 1 / 4:
            nominal = rates['3 Mo'].mean()
        elif 1 / 4 < self.year_time <= 1 / 3:
            nominal = rates['4 Mo'].mean()
        elif 1 / 3 < self.year_time <= 1 / 2:
            nominal = rates['6 Mo'].mean()
        elif 1 /  2 < self.year_time <= 1:
            nominal = rates['1 Yr'].mean()
        elif 1 < self.year_time<= 2:
            nominal = rates['2 Yr'].mean()
        elif 2 < self.year_time <= 3:
            nominal = rates['3 Yr'].mean()
        elif 3 < self.year_time <= 5:
            nominal = rates['5 Yr'].mean()
        elif 5 < self.year_time <= 7:
            nominal = rates['7 Yr'].mean()
        elif 7 < self.year_time <= 10:
            nominal = rates['10 Yr'].mean()
        elif 10 < self.year_time <= 20:
            nominal = rates['20 Yr'].mean()
        elif 20 < self.year_time <= 30:
            nominal = rates['30 Yr'].mean()

        return (1 + nominal) / (1 + 0.077) - 1

    def benchmark_rate(self): # calculate performance of a benchmark
        # set the S&P as the default benchmark
        gspc = yf.Ticker('^GSPC').info
        
        #initial time time - (difference)
        print(type((self.final_time - self.initial_time)))
        '''print(self.initial_time - (self.year_time * 365))
        hist_table = yf.download(gspc, self.initial_time-(self.year_time*365))['Adj Close']
        hist_s = hist_table.loc[self.initial_time-(self.year_time*365)][gspc]
        benchmark_rate = abs(gspc['regularMarketPrice']- hist_s)/hist_s
        return benchmark_rate'''

    def pricing(self,time:float  = 1/52, price: float = .050): # pricing of a singular futures contract on underlying equity
        if self.div_rate() is None:
            div_rate = 0
        else:
            div_rate = self.div_rate()
            
        return 100 * price * np.e**(self.risk_free_rate() - (div_rate*time))

    def beta(self): # covariance function
        #get the historical data beginning to end
        historical_data = self.get_historical_data()
        hist_price = [price for price in historical_data["Close"]]
        future_price = []
        for i in range(len(hist_price)):
            future_price.append(self.pricing(time = 1/52, price = hist_price[i]))
        return np.cov(hist_price,future_price)[0,1]
            


    def expected_return(self): # calculate the expected return (w/ respect to a benchmark)
        return float(self.risk_free_rate() + self.beta()*(self.benchmark_rate() - self.risk_free_rate()))

    def n(self): # calculate the ideal number of futures contracts on the underlying equity
        return float(self.beta() * self.shares / self.pricing(1/52,self.spot_price))

    def portfolio(self): # construct an ideal portfolio, containing the user's existing
        # position in the underlying equity and n futures contracts
        data = [("stock", self.shares / self.spot_price, "$" + str(self.shares)),
                    ("futures", self.n(), "$" + str(self.n() * self.pricing(1/52,self.spot_price)))]

        portfolio = pd.DataFrame(data, columns = ["Instrument", " Qty", "Value"])

        return portfolio

    def basis_risk(self): # calculate basis risk of portfolio
        return float(self.n() * (self.spot_price - self.pricing(1/52,self.spot_price)))

    def metrics(self): # calculate performance & risk metric of portfolio
        """"
            add implementation
        """
        pass;

    def _str_(self):
        return f"risk-free-rate{self.risk_free_rate()}, benchmark rate {self.benchmark_rate()}, pricing {self.pricing(1/52,self.spot_price)}, beta {self.beta()}, expect_return {self.expected_return()}, number of future contracts (100 share per contract) {self.n()}, protfolio {self.portfolio()}, basis risk {self.basis_risk()}, metrics {self.metrics()}"
        pass



In [638]:
#practicing with microsoft stock 
 #stock1 = Edward_Dashboard("MSFT", s: float,  tf: str, tol: float = 0.00, cap: float = 10000, shares: float = 100)


In [639]:
msft= Edward_Dashboard("msft","2022-11-27",300)

2022-11-27 00:00:00
0.005479452054794521


In [598]:
msft.ticker_symbol

'msft'

In [599]:
msft.spot_price

247.49

In [600]:
msft.cap

10000.0

In [601]:
msft.initial_time

datetime.datetime(2022, 11, 25, 0, 0)

In [602]:
msft.final_time

datetime.datetime(2022, 11, 27, 0, 0)

In [603]:
msft.year_time

0.005479452054794521

In [604]:
msft.risk_tolerance

300.0

In [605]:
msft.liquid

1000.0

In [606]:
msft.tickerObj

yfinance.Ticker object <MSFT>

In [607]:
msft.div_rate()

0.68

In [608]:
msft.risk_free_rate()

1.2135393690366656

In [609]:
msft.get_historical_data()

,Date,Close
0,1986-03-13 00:00:00-05:00,0.060809
1,1986-03-14 00:00:00-05:00,0.062980
2,1986-03-17 00:00:00-05:00,0.064067
3,1986-03-18 00:00:00-05:00,0.062437
4,1986-03-19 00:00:00-05:00,0.061351
...,...,...
9248,2022-11-18 00:00:00-05:00,241.220001
9249,2022-11-21 00:00:00-05:00,242.050003
9250,2022-11-22 00:00:00-05:00,245.029999
9251,2022-11-23 00:00:00-05:00,247.580002


In [633]:
msft.beta()


1461335.8348738

In [588]:
msft.pricing(1/52, .0608085505664348 )

20.198488328136627

In [642]:
msft.benchmark_rate()




<class 'datetime.timedelta'>


In [565]:
help(msft)

Help on Edward_Dashboard in module __main__ object:

class Edward_Dashboard(builtins.object)
 |  Edward_Dashboard(ticker: str, final_time: str, r_tol: float = 0.0, liquid: float = 1000, cap: float = 10000, shares: float = 100)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, ticker: str, final_time: str, r_tol: float = 0.0, liquid: float = 1000, cap: float = 10000, shares: float = 100)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  basis_risk(self)
 |  
 |  benchmark_rate(self)
 |  
 |  beta(self)
 |  
 |  div_rate(self)
 |  
 |  expected_return(self)
 |  
 |  get_historical_data(self)
 |  
 |  metrics(self)
 |      "
 |      add implementation
 |  
 |  n(self)
 |  
 |  portfolio(self)
 |  
 |  pricing(self, time: float, price: float)
 |  
 |  risk_free_rate(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined

In [590]:
msft.n()

0.0608085617423057 20.19849204037275
0.0629801824688911 20.919829015033127
0.064066581428051 21.280693013465665
0.0624372959136962 20.73950095842822
0.0613514669239521 20.37882628403786
0.0597227402031421 19.837819840806436
0.0580940432846546 19.29682329687137
0.0564653575420379 18.755830465172394
0.0575511381030082 19.11648905320626
0.0591798424720764 19.65748807196544
0.0602656565606594 20.01815779670767
0.0597227402031421 19.837819840806436
0.0591798424720764 19.65748807196544
0.0597227402031421 19.837819840806436
0.0602656565606594 20.01815779670767
0.0602656565606594 20.01815779670767
0.0591798424720764 19.65748807196544
0.0597227402031421 19.837819840806436
0.0608085617423057 20.19849204037275
0.0613514669239521 20.37882628403786
0.0624372959136962 20.73950095842822
0.0629801824688911 20.919829015033127
0.0629801824688911 20.919829015033127
0.065152384340763 21.64135902597174
0.0656952932476997 21.821694507048893
0.0635236874222755 21.10036248203668
0.0635236874222755 21.10036248

0.5831120610237122 193.68957241819822
0.5722537636756897 190.08282319947025
0.5711671113967896 189.72187505701902
0.5570511221885681 185.03303375745767
0.5624808669090271 186.83660635274808
0.5733394026756287 190.4434347658466
0.5472782850265503 181.78683670932813
0.5168741941452026 171.6876537971558
0.5038437843322754 167.35940426537815
0.4951567947864532 164.4738880786992
0.4908128976821899 163.03099634485937
0.4745252132415771 157.62079332239685
0.4908128976821899 163.03099634485937
0.5309901237487793 176.3764752981245
0.5385911464691162 178.90127102616756
0.5559654235839844 184.6724023924887
0.5581369996070862 185.39372451820458
0.5679098963737488 188.63994136492678
0.5624808669090271 186.83660635274808
0.5505358576774597 182.8688891929762
0.5429352521896362 180.34423205508162
0.554879367351532 184.31165223596386
0.5537938475608826 183.9510802667728
0.5516215562820435 183.2295205579452
0.5646524429321289 187.55792847846396
0.570081353187561 189.36122389345738
0.5581369996070862 185

2.0474085807800293 680.0780143021391
2.101157665252685 697.931593202739
2.0718393325805664 688.1930614541285
2.042521715164185 678.4547672886297
2.066953420639038 686.5701312181012
2.130476713180542 707.6703625344617
2.096271514892578 696.3085837723413
2.0327484607696533 675.2084316503515
2.042521715164185 678.4547672886297
2.0547375679016118 682.5124492533655
2.066953420639038 686.5701312181012
2.0938289165496826 695.4972374458836
2.12070369720459 704.4241060905542
2.0938289165496826 695.4972374458836
2.0718393325805664 688.1930614541285
2.0913853645324707 694.6855743419435
2.057180404663086 683.3238747741938
2.01320219039917 668.715839568166
1.959452986717224 650.8622210703804
1.9350194931030276 642.7462631831297
2.027862310409546 673.5854222199536
2.0058722496032715 666.2810878394574
2.0083162784576416 667.0929093321387
2.018089056015014 670.3390865816754
1.964338541030884 652.4850325148522
1.9203611612319944 637.8772744891212
1.888599157333374 627.3270400393852
1.9692257642745967 6

10.344537734985352 3436.096120590708
10.466696739196776 3476.6730986268067
10.412946701049805 3458.819202948725
10.549766540527344 3504.266001219724
10.564424514770508 3509.1348711221767
10.67681121826172 3546.4658302974813
10.86738395690918 3609.7674745790086
10.862497329711914 3608.1443067598693
10.67192554473877 3544.8429792558236
10.69147491455078 3551.336600864826
10.686585426330566 3549.7124827132407
10.613289833068848 3525.366232536084
10.469141006469728 3477.48499931386
10.339651107788086 3434.472952771569
10.36896800994873 3444.2110093539563
10.828289031982422 3596.7814984709316
10.608402252197266 3523.7427479394632
10.603519439697266 3522.1208472302533
10.583971977233888 3515.6278591762157
10.075791358947754 3346.827909310225
10.427608489990234 3463.6893399611067
10.232148170471191 3398.7642110829574
10.05623722076416 3340.332703813811
10.163740158081056 3376.041445502423
10.486246109008787 3483.1667202358094
10.496017456054688 3486.412422319124
10.442265510559082 3468.557893

20.74027442932129 6889.198757094432
21.10305595397949 7009.702178458943
20.824722290039062 6917.249403149225
20.47446060180664 6800.904636571922
20.57766532897949 6835.185662147953
20.69963264465332 6875.698967910118
20.787193298339844 6904.783575667032
21.09367179870605 7006.58508803343
20.91854286193848 6948.413338299746
20.68086051940918 6869.463519949163
20.749670028686523 6892.319648849732
20.2837028503418 6737.541537458836
20.33061408996582 6753.123821811576
20.490095138549805 6806.097886616217
20.58704376220703 6838.300851908573
20.23053741455078 6719.881826377382
19.76456451416016 6565.1018143215915
20.20864486694336 6712.609882494447
19.351762771606445 6427.983414012144
19.60819625854492 6513.161711219069
19.007753372192383 6313.7154406085865
18.970233917236328 6301.252780901216
18.48862648010254 6141.27951880113
19.404932022094727 6445.644392203525
19.486236572265625 6472.650939676765
18.995250701904297 6309.562487816059
18.842012405395508 6258.662048410341
17.806869506835938

...

20.846240997314453 6924.397170259111
20.43648910522461 6788.291824337956
20.919416427612305 6948.703506473502
20.816972732543945 6914.67526932832
21.0218505859375 6982.728575843861
21.08039093017578 7002.173644815373
20.838924407958984 6921.966853415154
21.153562545776367 7026.478713919835
20.809659957885746 6912.2462195942935
20.838924407958984 6921.966853415154
21.168195724487305 7031.339347607748
21.168195724487305 7031.339347607748
21.33649253845215 7087.2417034584705
21.24868392944336 7058.074733556166
21.0218505859375 6982.728575843861
21.168195724487305 7031.339347607748
20.977951049804688 6968.1466747801205
20.79502296447754 6907.384318796449
20.963319778442383 6963.286674647171
21.2779483795166 7067.795367377029
21.62916755676269 7184.458178546528
21.58526992797852 7169.876911037754
21.57794761657715 7167.4446935289025
21.555997848510746 7160.153742997034
21.782825469970703 7235.498000044444
21.73160552978516 7218.484515026819
21.548677444458008 7157.722159043146
21.7389259338

20.534236907958984 6820.760249160453
20.30342483520508 6744.092495792129
20.693422317504883 6873.636112945592
20.86850929260254 6931.793924470058
20.48648452758789 6804.898567068351
20.605867385864254 6844.553405853569
20.725255966186523 6884.210145303683
21.091371536254883 7005.8210207462025
21.481359481811523 7135.361470124843
21.584827423095703 7169.729926285981
21.656457901000977 7193.523082979098
21.47340583801269 7132.7195459227305
20.68545913696289 6870.991020968657
19.94526481628418 6625.124178106684
19.94526481628418 6625.124178106684
20.247709274291992 6725.585721723091
20.43077087402344 6786.392426554282
20.359140396118164 6762.5992698611635
20.255672454833984 6728.2308137000255
19.80997085571289 6580.184223807313
19.523441314697266 6485.009062814987
20.168119430541992 6699.148740162969
20.605867385864254 6844.553405853569
20.964025497436523 6963.5210899840495
20.89238739013672 6939.725399071074
21.441560745239254 7122.141712234851
21.48931884765625 7138.005294991848
21.4574

41.510536193847656 13788.358265341287
41.82903671264648 13894.153074649148
40.289642333984375 13382.819732542346
40.793922424316406 13550.32406250586
40.64352416992188 13500.36698644234
40.121559143066406 13326.98833484765
40.11270523071289 13324.04737270228
40.95318222045898 13603.224634934613
41.47515106201172 13776.60455363923
41.31591033935547 13723.710316760122
41.41322326660156 13756.034291050704
42.05905532836914 13970.55726915223
42.09445190429688 13982.314782184076
41.24512100219727 13700.196557806366
41.35130310058594 13735.466562682035
41.87328338623047 13908.850282716436
41.05934143066406 13638.487037150713
41.35130310058594 13735.466562682035
41.34245300292969 13732.526867646591
41.58131408691406 13811.868222965257
41.86441802978516 13905.90551924128
42.095970153808594 13982.819091935842
41.50820159912109 13787.582794064692
40.66218948364258 13506.566955325277
38.35568618774414 12740.426676251944
37.117820739746094 12329.25077137857
36.04027557373047 11971.327695494232
38.

162.082275390625 53838.10199683417
160.44898986816406 53295.581278043166
157.76576232910156 52404.306290814566
160.8573455810547 53431.222861730275
163.36553955078125 54264.35777697341
167.9737091064453 55795.03163973002
165.49459838867188 54971.557170586115
169.52920532226562 56311.71345284743
175.10951232910156 58165.297609928304
174.8956298828125 58094.25329042519
178.52183532714844 59298.75278368024
178.77462768554688 59382.721624459875
183.4508056640625 60935.985523031944
179.309326171875 59560.329888997796
179.57180786132812 59647.51718899726
178.59963989257812 59324.59675779356
180.1940155029297 59854.192955308055
182.0217437744141 60461.300801403646
182.567626953125 60642.62423225359
179.77957153320312 59716.529064174145
174.0962677001953 57828.732939008376
166.59002685546875 55335.420457817934
163.8410186767578 54422.29542697245
165.8881378173828 55102.27729930872
154.1998291015625 51219.82713443883
157.93345642089844 52460.008443296494
168.44219970703125 55950.647944339
160.3

439942.4018071291